## Importing

## Libs

In [1]:
import os

import cv2 as cv
import numpy as np
import seaborn as sns

### Utils

In [ ]:
def read_image(name: str, itype: str) -> np.ndarray:
    path = os.path.join(GetDirectory(), "images", itype, name)
    #print(f"Read on: {path}")

    if not os.path.isfile(path):
        print(f"Image file '{path}' does not exist.")
        return

    image = cv.imread(path, cv.IMREAD_UNCHANGED)

    return image

In [ ]:
def normalize_image(img: np.ndarray, depth: int = 8) -> np.ndarray:
    N = np.float32((2 ** depth) - 1)

    def normalize(n: np.float32):
        return n / N

    normalizev = np.vectorize(normalize)

    norm = img.astype(np.float32)
    norm = normalizev(norm)

    return norm

In [ ]:
def denormalize_image(norm, depth=8):
    max_value = np.float32((2 ** depth) - 1)

    denormalizev = np.vectorize(lambda n: np.floor(n * max_value))

    denorm = denormalizev(norm)

    if depth == 8:
        denorm = denorm.astype(np.uint8)
    elif depth == 16:
        denorm = denorm.astype(np.uint16)
    elif depth == 32:
        denorm = denorm.astype(np.uint32)
    elif depth <= 8:
        if depth == 0: max_value = np.float32(1)
        frac = 255 / max_value
        denorm = (denorm * frac).astype(np.uint8)
    else:
        assert False, "Unsupported depth"

    return denorm

## Ops

### Noise

In [ ]:
def put_noise(image: np.ndarray, standard_deviation: float=0.1, mean: float=0.0) -> np.ndarray:
    print("\nNoise")

    image_noise = np.random.normal(mean, standard_deviation, image.shape)
    image_noise += image
    image_noise = np.clip(image_noise, 0.0, 1.0)

    return image_noise

### Histogram

In [ ]:
def img_8bit_val_count_percent(img: np.ndarray):
    counts = np.zeros(256, np.uint64)

    h, w = img.shape
    for row in range(h):
        for col in range(w):
            idx = img[row, col]
            counts[idx] += 1

    total = img.size
    counts = counts / total

    return counts

In [ ]:
def make_histogram(image_name: str, title: str = ''):
    img = read_image(image_name, "manipulations/mono")

    if title == '':
        title = image_name

    try:
        plt.xlabel('Pixel Value [0, 255]')
        plt.ylabel('% of Pixels')
        vals = img_8bit_val_count_percent(img)
        plt.bar(range(256), vals, width=1, align='edge')  # Adjusted the range and width
        plt.xlim(0, 255)  # Set x-axis limit
        plt.ylim(0)  # Set y-axis limit to start from 0, but don't specify an upper limit
        write_plot_image(image_name, "histogram", plt)
    finally:
        plt.clf()

In [ ]:

def plot_histogram(image: np.ndarray, image_noise: np.ndarray, image_mask: np.ndarray, image_name: str) -> np.ndarray:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))  # Create two subplots side by side
# Use seaborn color palette for better colors
    colors = sns.color_palette("bright")

    # Plot a histogram of image without noise (excluding 255 and 0) in ax1
    ax1.hist(image.ravel(), bins=256, range=(0, 255), color=colors[0], alpha=0.7, label='Without Noise')
    ax1.hist(image_noise.ravel(), bins=256, range=(0, 255), color=colors[3], alpha=0.7, label='With Noise')

    # Set labels and legend for ax1
    ax1.set_xlabel('Pixel Value')
    ax1.set_ylabel('Frequency')
    ax1.legend(loc='upper right')

    # Plot a histogram of image_mask in ax2
    ax2.hist(image_noise.ravel(), bins=255, range=(0, 255), color=colors[3], alpha=0.7, label='With Noise')
    ax2.hist(image_mask.ravel(), bins=255, range=(0, 255), color=colors[2], alpha=0.7, label='With Mask')

    # Set labels and legend for ax2
    ax2.set_xlabel('Pixel Value')
    ax2.set_ylabel('Frequency')
    ax2.legend(loc='upper right')

    # Save the plot

    write_plot_image(image_name, "histogram", plt)

### Blur

In [ ]:
def blur(image: np.ndarray) -> np.ndarray:
    print("\nBlur")
    mask = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]]) * (1/9)

    res = convolution(image, mask).clip(0.0,1.0)
    return res

### Calculate

In [ ]:
def calculate_mse(image_without_noise: np.ndarray, image_with_noise: np.ndarray) -> float:
    n, m = image_without_noise.shape[:2]

    mse = np.sum((image_without_noise - image_with_noise)**2) / (n * m)

    return mse


def calculate_psnr(image_without_noise: np.ndarray, mse_value: float) -> float:
    max_value = np.max(image_without_noise)

    psnr = 10 * np.log10((max_value ** 2) / mse_value)

    return psnr

### Convulation

In [ ]:
def valid_mask_shape(mask: np.ndarray) -> bool:
  is_odd = lambda x: (x % 2) != 0
  return (mask.shape[0] == mask.shape[1]) and is_odd(mask.shape[0])


def mat_bounds_check(row, col, height, width) -> bool:
  ok = (col < width) and (col > 0) and (row < height) and (row > 0)
  return ok

In [ ]:
def convolution(img: np.ndarray, mask: np.ndarray) -> np.ndarray:
  assert valid_mask_shape(mask)

  N = mask.shape[0] // 2
  border_img = cv.copyMakeBorder(img, N,N,N,N, cv.BORDER_CONSTANT, value=(0,0,0))
  res = np.zeros( img.shape )
  HEIGHT, WIDTH = border_img.shape

  for row in range(N, HEIGHT - N):
    for col in range(N, WIDTH - N):
      slice = border_img[(row-N):(row+N+1), (col-N):(col+N+1)]
      total = np.clip(0, np.sum(mask * slice), 1.0)
      res[row - N, col - N] = total


  return res

In [ ]:
images_name = ["barb.tif", "tank.tiff", "lena2.tif"]

    std_dev = float(sys.argv[1])

    for image_name in images_name:
        print(image_name)

        image = read_image(image_name, "repository")
        image_normalized = normalize_image(image)

        image_noise = put_noise(image_normalized, std_dev)
        image_noise_desnormalized = denormalize_image(image_noise)
        write_image(image_name, "noise", image_noise_desnormalized)

        mse = calculate_mse(image, image_noise_desnormalized)
        psnr = calculate_psnr(image, mse)

        print("\nMSE")
        print(mse)
        print("\nPSNR")
        print(psnr)

        image_blur = blur(image_noise)
        image_blur_desnormalized = denormalize_image(image_blur)
        write_image(image_name, "mask", image_noise_desnormalized)

        plot_histogram(image, image_noise_desnormalized, image_blur_desnormalized, image_name)